In [ ]:
import qubic
import scipy
from pyoperators import *
from time import time
import healpy as hp
import pysm3
import pysm3.units as u

from non_linear_pcg import non_linear_pcg

rc('figure',figsize=(10,8))
rc('font',size=13)

# Read me
A non-linear PCG is not as straightforward as a linear PCG. It can be very badly used so be aware of how it works.

**Conjugation method**

Many methods can be used to conjugate the iterative search directions of the non-linear PCG. Here, five possiblities are implemented: 'polak-ribiere', 'fletcher-reeves', 'hestenes-stiefel', 'dai-yuan', 'hybrid' (hybrid version of hestenes-stiefel and dai-yuan).
The fastest one will depend on the problem you are trying to solve. You should test them to make the best choice. The most widly used is polak-ribiere.
For more details, see: Hager, W. W., & Zhang, H. (2006). A survey of nonlinear conjugate gradient methods. Pacific journal of Optimization, 2(1), 35-58. https://people.clas.ufl.edu/hager/files/cg_survey.pdf

Other possibilities could be easily implemented.


**Line search**

In a PCG, at each iteration, one needs to find:
$$\alpha = argmin f(x+\alpha d)$$
with x the current estimation of the solution, and d the current search direction. It is the size of the step we will do in the direction d.

For the linear PCG, we have an analytic solution to this small problem. This is generally not the case for a non-linear gradient. We perform a secant method. We take a first guess for alpha, which we call sigma_0. Then we iterate and form a quadratic which has the same derivative as f at the points alpha and the previous alpha. We find the minimum of this quadratic, update x and iterate.

If you have an analytical formula for this small problem, you should change the code and remove this secant method. This will improve the speed of the computation.

The initial guess sigma_0 is very important and has to be chosen by hand. If it is chosen poorly, the PCG will not converge or very slowly.

There is also a tolerance parameter for this line search, 'tol_linesearch'. You can play with it. If it is too big, the PCG won't converge. If it's too small, the algorithm will be unnecessarily slower. There is a maximum iteration parameter 'maxiter_linesearch'. By default, it is set to 10. If it is not enough, it is probably because you have a poor initial guess sigma_0, or that your tolerance is too low.


**Initial guess for x**

As the gradient is not linear, the function f could have many local minima. The algorithm will converge to the nearest local minimum, it can even converge to a local maximum! So you should define with great care the initial guess for x. It is the parameter 'x0'.


**Preconditioning**

Preconditioning is implemented. M should approximate the inverse of the Hessian matrix of f. Make sure that it is not computationaly too expensive to apply M on a vector.


**More details**

The algorithm implemented is the B5 from: Shewchuk, J. R. (1994). An introduction to the conjugate gradient method without the agonizing pain. http://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf
We highly recommend to read to very well written article.

# Linear problem
First we will check that the non-linear PCG is giving the same result as the linear PCG for a linear gradient.

In [ ]:
nside = 16
ndetectors = 1
npointings = 1000

In [ ]:
######## Technological Demonstrator Configuration #######
#dictname = 'RealisticScanning-BmodesNoDustNoSystPaper0_2020.dict'
dictname = 'pipeline_demo.dict'
#dictname = 'global_source_oneDet.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictname)
d['hwp_stepsize'] = 3
d['npointings'] = npointings
d['nside'] = nside
d['synthbeam_kmax'] = 3
d['synthbeam_fraction'] = 0.95
d['random_pointing'] = True
d['repeat_pointing'] = False
#d['dtheta'] = 15
q = qubic.QubicInstrument(d)
s = qubic.QubicScene(d)
samp = qubic.get_pointing(d)

Creating a map, the operator H, and generating the TOD.

In [ ]:
acq = qubic.QubicAcquisition(q, samp, s, d)
H_op = acq[:ndetectors].get_operator()

In [ ]:
# We create a mask for Qubic's patch. Please don't look too much at this horrible code.
def circular_mask(nside, center, radius):
    lon = center[0]
    lat = center[1]
    vec = hp.ang2vec(lon, lat, lonlat=True)
    disc = hp.query_disc(nside, vec, radius=np.deg2rad(radius))
    m = np.zeros(hp.nside2npix(nside))
    m[disc] = 1
    return np.array(m, dtype=bool)

mask = circular_mask(nside, qubic.equ2gal(0, -57), d['dtheta'])

Mask = np.zeros((H_op.shape[1], np.count_nonzero(mask), 3))
a = 0
for i in range(len(mask)):
    if mask[i]:
        Mask[i*3:(i+1)*3,a] = np.identity(3)
        a += 1
Mask = np.reshape(Mask, (H_op.shape[1], np.count_nonzero(mask)*3))

PatchMask = ReshapeOperator(H_op.shape[1], H_op.shapein) * asoperator(Mask, flags='linear,real') * ReshapeOperator(
    (np.count_nonzero(mask), 3), np.count_nonzero(mask)*3)

H = H_op * PatchMask

In [ ]:
map = np.ones((np.count_nonzero(mask), 3))

tod = H(map)

invN = acq[:ndetectors].get_invntt_operator()

A = CompositionOperator((H.T, invN, H), flags='linear,real,symmetric,square')
b = H.T * invN * tod

In [ ]:
print(f'map: {map.shape}')
print(f'H: {H.shape}')
print(f'TOD: {tod.shape}')
print(f'A: {A.shape}')
print(f'b: {b.shape}')

In [ ]:
def _dot(x, y, comm):
    d = np.array(np.dot(x.ravel(), y.ravel()))
    if comm is not None:
        comm.Allreduce(MPI.IN_PLACE, d)
    return d

def grad_chi2(x, out):
    out[...] = A(x) - b

grad_f = Operator(grad_chi2, shapein=A.shapein, shapeout=A.shapeout, dtype='float64')

Computing the PCG

In [ ]:
Max_iterations = [0,1,2,3,4,5,7,9,11,13,15,18,21,24,27,30,33,36,39,42,45,48,51]

residues_cg = []
residues_PR_cg = []
residues_FR_cg = []
residues_HS_cg = []
residues_DY_cg = []
residues_hybrid_cg = []

start = time()
for i, val in enumerate(Max_iterations):
    x_cg = pcg(A, b, tol=1e-50, maxiter=val, M=None)['x']
    residues_cg.append(np.linalg.norm(b-A(x_cg))/np.linalg.norm(b))
print(f'time for linear CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_PR_cg = non_linear_pcg(grad_f, conjugate_method='polak-ribiere', tol=1e-50, sigma_0=10, tol_linesearch=1e-3, maxiter=val)['x']
    residues_PR_cg.append(np.linalg.norm(b-A(x_PR_cg))/np.linalg.norm(b))
print(f'time for PR CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_FR_cg = non_linear_pcg(grad_f, conjugate_method='fletcher-reeves', tol=1e-50, sigma_0=10, tol_linesearch=1e-3, maxiter=val)['x']
    residues_FR_cg.append(np.linalg.norm(b-A(x_FR_cg))/np.linalg.norm(b))
print(f'time for FR CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_HS_cg = non_linear_pcg(grad_f, conjugate_method='hestenes-stiefel', tol=1e-50, sigma_0=10, tol_linesearch=1e-3, maxiter=val)['x']
    residues_HS_cg.append(np.linalg.norm(b-A(x_HS_cg))/np.linalg.norm(b))
print(f'time for HS CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_DY_cg = non_linear_pcg(grad_f, conjugate_method='dai-yuan', tol=1e-50, sigma_0=10, tol_linesearch=1e-3, maxiter=val)['x']
    residues_DY_cg.append(np.linalg.norm(b-A(x_DY_cg))/np.linalg.norm(b))
print(f'time for DY CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_hybrid_cg = non_linear_pcg(grad_f, conjugate_method='hybrid', tol=1e-50, sigma_0=10, tol_linesearch=1e-3, maxiter=val)['x']
    residues_hybrid_cg.append(np.linalg.norm(b-A(x_hybrid_cg))/np.linalg.norm(b))
print(f'time for hybrid CG: {time()-start}')

In [ ]:
plt.plot(Max_iterations, residues_cg, label='linear CG')
plt.plot(Max_iterations, residues_PR_cg, label='Polak-Ribière CG')
plt.plot(Max_iterations, residues_FR_cg, label='Fletcher-Reeves CG')
plt.plot(Max_iterations, residues_HS_cg, label='Hestenes-Stiefel CG')
plt.plot(Max_iterations, residues_DY_cg, label='Dai-Yuan CG')
plt.plot(Max_iterations, residues_hybrid_cg, label='hybrid CG')
plt.yscale('log')
plt.grid(axis='y', linestyle='dotted')
plt.xlabel('Number of iterations')
plt.ylabel(r'Relative residue $\frac{||b-Ax||}{||b||}$')
plt.title(f'Solving $Ax=b$, A has size {A.shape}')
plt.legend()
plt.show()

It works exactly as the linear PCG! Each iteration takes a bit more time because the algorithm is not as efficient as for the specific linear case.

# Non-linear gradient
Let's test the non-linear PCG with a very simple non-linear gradient. We set the tolerance to $10^{-16}$ to avoid problems when the residue becomes smaller than the computer precision.

In [ ]:
N = 100
betas = 1+np.random.random(N)
dust_power = 0.5**betas

def grad_chi2_spectral_indexes(x, out):
    power = 0.5**x
    out[...] = 2*log(0.5)*(power - dust_power) * power

grad_beta = Operator(grad_chi2_spectral_indexes, shapein=N, shapeout=N, dtype='float64')

In [ ]:
Max_iterations = [0,1,2,3,4,5,7,9,11,13,15,18,21,24,27,30,33,36,39]

sigma=1

residues_PR_cg = []
residues_FR_cg = []
residues_HS_cg = []
residues_DY_cg = []
residues_hybrid_cg = []

start = time()
for i, val in enumerate(Max_iterations):
    x_PR_cg = non_linear_pcg(grad_beta, conjugate_method='polak-ribiere', tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=val)['x']
    residues_PR_cg.append(np.linalg.norm(grad_beta(x_PR_cg))/np.linalg.norm(grad_beta(np.zeros(N))))
print(f'time for PR CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_FR_cg = non_linear_pcg(grad_beta, conjugate_method='fletcher-reeves', tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=val)['x']
    residues_FR_cg.append(np.linalg.norm(grad_beta(x_FR_cg))/np.linalg.norm(grad_beta(np.zeros(N))))
print(f'time for FR CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_HS_cg = non_linear_pcg(grad_beta, conjugate_method='hestenes-stiefel', tol=1e-16, sigma_0=sigma, tol_linesearch=1e-6, maxiter=val)['x']
    residues_HS_cg.append(np.linalg.norm(grad_beta(x_HS_cg))/np.linalg.norm(grad_beta(np.zeros(N))))
print(f'time for HS CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_DY_cg = non_linear_pcg(grad_beta, conjugate_method='dai-yuan', tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=val)['x']
    residues_DY_cg.append(np.linalg.norm(grad_beta(x_DY_cg))/np.linalg.norm(grad_beta(np.zeros(N))))
print(f'time for DY CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_hybrid_cg = non_linear_pcg(grad_beta, conjugate_method='hybrid', tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=val)['x']
    residues_hybrid_cg.append(np.linalg.norm(grad_beta(x_hybrid_cg))/np.linalg.norm(grad_beta(np.zeros(N))))
print(f'time for hybrid CG: {time()-start}')


In [ ]:
plt.plot(Max_iterations, residues_PR_cg, label='Polak-Ribière CG')
plt.plot(Max_iterations, residues_FR_cg, label='Fletcher-Reeves CG')
plt.plot(Max_iterations, residues_HS_cg, label='Hestenes-Stiefel CG')
plt.plot(Max_iterations, residues_DY_cg, label='Dai-Yuan CG')
plt.plot(Max_iterations, residues_hybrid_cg, label='hybrid CG')
plt.yscale('log')
plt.grid(axis='y', linestyle='dotted')
plt.xlabel('Number of iterations')
plt.ylabel(r'Relative residue $\frac{||\nabla \chi^2(\vec{\beta})||}{||\nabla \chi^2(\vec{0})||}$')
plt.title(r'$\chi^2$ minimization using non-linear PCG')
plt.legend()
plt.show()

This works great, but let's put sigma_0 = 100 instead of 1:

In [ ]:
Max_iterations = [0,1,2,3,4,5,7,9,11,13,15,18,21,24,27,30,33,36,39]

sigma=100

residues_PR_cg = []
residues_FR_cg = []
residues_HS_cg = []
residues_DY_cg = []
residues_hybrid_cg = []

start = time()
for i, val in enumerate(Max_iterations):
    x_PR_cg = non_linear_pcg(grad_beta, conjugate_method='polak-ribiere', tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=val)['x']
    residues_PR_cg.append(np.linalg.norm(grad_beta(x_PR_cg))/np.linalg.norm(grad_beta(np.zeros(N))))
print(f'time for PR CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_FR_cg = non_linear_pcg(grad_beta, conjugate_method='fletcher-reeves', tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=val)['x']
    residues_FR_cg.append(np.linalg.norm(grad_beta(x_FR_cg))/np.linalg.norm(grad_beta(np.zeros(N))))
print(f'time for FR CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_HS_cg = non_linear_pcg(grad_beta, conjugate_method='hestenes-stiefel', tol=1e-16, sigma_0=sigma, tol_linesearch=1e-6, maxiter=val)['x']
    residues_HS_cg.append(np.linalg.norm(grad_beta(x_HS_cg))/np.linalg.norm(grad_beta(np.zeros(N))))
print(f'time for HS CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_DY_cg = non_linear_pcg(grad_beta, conjugate_method='dai-yuan', tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=val)['x']
    residues_DY_cg.append(np.linalg.norm(grad_beta(x_DY_cg))/np.linalg.norm(grad_beta(np.zeros(N))))
print(f'time for DY CG: {time()-start}')

start = time()
for i, val in enumerate(Max_iterations):
    x_hybrid_cg = non_linear_pcg(grad_beta, conjugate_method='hybrid', tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=val)['x']
    residues_hybrid_cg.append(np.linalg.norm(grad_beta(x_hybrid_cg))/np.linalg.norm(grad_beta(np.zeros(N))))
print(f'time for hybrid CG: {time()-start}')


In [ ]:
plt.plot(Max_iterations, residues_PR_cg, label='Polak-Ribière CG')
plt.plot(Max_iterations, residues_FR_cg, label='Fletcher-Reeves CG')
plt.plot(Max_iterations, residues_HS_cg, label='Hestenes-Stiefel CG')
plt.plot(Max_iterations, residues_DY_cg, label='Dai-Yuan CG')
plt.plot(Max_iterations, residues_hybrid_cg, label='hybrid CG')
plt.yscale('log')
plt.grid(axis='y', linestyle='dotted')
plt.xlabel('Number of iterations')
plt.ylabel(r'Relative residue $\frac{||\nabla \chi^2(\vec{\beta})||}{||\nabla \chi^2(\vec{0})||}$')
plt.title(r'$\chi^2$ minimization using non-linear PCG')
plt.legend()
plt.show()

The PCG converges in one step! This is because our function is very simple. It is equivalent to the linear case where the matrix A has only one eigenvalue. CG converges in just one step.
But this shows the importance of well choosing the parameter sigma_0.

In [ ]:
plt.plot(Max_iterations, residues_PR_cg, label='Polak-Ribière CG')
plt.plot(Max_iterations, residues_FR_cg, label='Fletcher-Reeves CG')
plt.plot(Max_iterations, residues_HS_cg, label='Hestenes-Stiefel CG')
plt.plot(Max_iterations, residues_DY_cg, label='Dai-Yuan CG')
plt.plot(Max_iterations, residues_hybrid_cg, label='hybrid CG')
plt.yscale('log')
plt.grid(axis='y', linestyle='dotted')
plt.xlabel('Number of iterations')
plt.ylabel(r'Relative residue $\frac{||\nabla \chi^2(\vec{\beta})||}{||\nabla \chi^2(\vec{0})||}$')
plt.title(r'$\chi^2$ minimization using non-linear PCG')
plt.legend()
plt.show()

# Fitting the spectral indexes

We consider the $\chi^2$ for the spectral indexes parameters:
$$\chi^2(\vec{\beta}) = \frac{1}{2}\left(d-\sum_{\nu}H_{\nu}A_{\nu}(\vec{\beta})c\right)^T N^{-1} \left(d-\sum_{\nu}H_{\nu}A_{\nu}(\vec{\beta})c\right)$$
Then, the gradient of $\chi^2$ is:
$$\nabla \chi^2(\vec{\beta}) = \left(\sum_{\nu}(J_{A_{\nu}\cdot c}(\vec{\beta}))^T H_{\nu}^T\right) N^{-1} \left(\sum_{\nu} H_{\nu} A_{\nu}(\vec{\beta}) c\right) - \left(\sum_{\nu}(J_{A_{\nu}\cdot c}(\vec{\beta}))^T H_{\nu}^T\right) N^{-1} d$$

with $J_{A_{\nu}\cdot c}(\vec{\beta})$ the Jacobian of the operator $\vec{\beta} \rightarrow A_{\nu}(\vec{\beta})c$ evaluated at the point $\vec{\beta}$.

We define some parameters

In [ ]:
nside = 16
ndetectors = 1
npointings = 8000
nside_beta = 8
frequencies = [140e9, 160e9, 200e9, 240e9]

The operator H

In [ ]:
H_list = []

for freq in frequencies:

    dictname = 'pipeline_demo.dict'
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(dictname)
    d['hwp_stepsize'] = 3
    d['npointings'] = npointings
    d['nside'] = nside
    d['filter_nu'] = freq
    d['synthbeam_kmax'] = 3
    d['synthbeam_fraction'] = 0.95
    d['random_pointing'] = True
    d['repeat_pointing'] = False
    d['dtheta'] = 180
    q = qubic.QubicInstrument(d)
    s = qubic.QubicScene(d)
    samp = qubic.get_pointing(d)

    acq = qubic.QubicAcquisition(q, samp, s, d)
    H_list.append(acq[:ndetectors].get_operator())

The spectra of the dust

In [ ]:
def modified_black_body(freq, beta):
    nu0 = frequencies[-1]
    return (np.exp(freq * 2.4e-12) - 1) / (np.exp(nu0 * 2.4e-12) - 1) * (freq/nu0)**beta

Definition of the map of the sky and the map of the spectral indexes that we will try to fit.

In [ ]:
component_maps = np.ones((12*nside**2, 6)) #IQU for CMB and dust

sky = pysm3.Sky(nside=nside_beta, preset_strings=['d1'])
true_beta = np.array(sky.components[0].mbb_index)

The operator $A_{\nu}(\vec{\beta})c$

In [ ]:
def function_A(betas, freq, out):
    power_beta = modified_black_body(freq, betas)
    A = hp.ud_grade(power_beta, nside)

    out[:,0] = component_maps[:,0] + A * component_maps[:,3]
    out[:,1] = component_maps[:,1] + A * component_maps[:,4]
    out[:,2] = component_maps[:,2] + A * component_maps[:,5]

A_list = []
for freq in frequencies:
    A_list.append(Operator(lambda betas, out, freq=freq : function_A(betas, freq, out), shapein=12*nside_beta**2, 
             shapeout=(12*nside**2, 3), dtype='float64'))


The operator $(J_{A_{\nu}\cdot c}(\vec{\beta}))^T$

In [ ]:
# Define the inner operator class
class GeneratedOperator(Operator):
    def __init__(self, beta, freq, **keywords):
        self.beta = beta
        self.freq = freq
        super().__init__(shapein=(12*nside**2,3), shapeout=12*nside_beta**2, dtype='float64', **keywords)
    
    def direct(self, input_vector, output):
        derive_power_beta = modified_black_body(self.freq, self.beta) * np.log(self.freq/frequencies[-1])
        productI = hp.ud_grade(component_maps[:,3]*input_vector[:,0], nside_beta) * (nside // nside_beta)**2
        productQ = hp.ud_grade(component_maps[:,4]*input_vector[:,1], nside_beta) * (nside // nside_beta)**2
        productU = hp.ud_grade(component_maps[:,5]*input_vector[:,2], nside_beta) * (nside // nside_beta)**2
        
        output[...] = derive_power_beta * (productI + productQ + productU)

# Define the outer operator class
class Transposed_Jacobian(Operator):
    def direct(self, beta, freq, output):
        # Create the generated operator
        generated_operator = GeneratedOperator(beta, freq)
        # Store the generated operator in the output
        output[...] = generated_operator

# Initialize the outer operator
transposed_jacobian = Transposed_Jacobian()

The operator $N^{-1}$

In [ ]:
invN = acq[:ndetectors].get_invntt_operator()

The TODs

In [ ]:
d = H_list[0] * A_list[0](true_beta)
for i in range(1, len(frequencies)):
    d = d + (H_list[i] * A_list[i](true_beta))

Finally, the gradient $\nabla \chi^2 (\vec{\beta})$

In [ ]:
def grad_operator(beta, out):
    W = H_list[0] * A_list[0](beta)
    for i in range(1, len(frequencies)):
        W = W + (H_list[i] * A_list[i](beta))
    W = invN * W - (invN(d))

    output_operator = np.empty((), dtype=object)
    transposed_jacobian.direct(beta, frequencies[0], output_operator)
    generated_operator = output_operator.item()
    X = generated_operator(H_list[0].T * W)
    for i in range(1, len(frequencies)):
        transposed_jacobian.direct(beta, frequencies[i], output_operator)
        generated_operator = output_operator.item()
        X = X + generated_operator(H_list[i].T * W)

    out[...] = X

grad_chi_squared = Operator(grad_operator, shapein=12*nside_beta**2, shapeout=12*nside_beta**2, dtype='float64')

Computation of the PCG

In [ ]:
Max_iterations = [0,1,2,3,4,5,7,9,11,13,15,18,21,24,27,30,33,36,39]
x0 = np.ones(12*nside_beta**2)

sigma=1e-2

residues_PR_cg = []

start = time()
for i, val in enumerate(Max_iterations):
    print(val)
    x_PR_cg = non_linear_pcg(grad_chi_squared, conjugate_method='polak-ribiere', x0=x0,  tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=val)['x']
    residues_PR_cg.append(np.linalg.norm(grad_chi_squared(x_PR_cg))/np.linalg.norm(grad_chi_squared(x0)))
print(f'time for PR CG: {time()-start}')


In [ ]:
plt.plot(Max_iterations, residues_PR_cg, label='Polak-Ribière CG')
plt.yscale('log')
plt.grid(axis='y', linestyle='dotted')
plt.xlabel('Number of iterations')
plt.ylabel(r'Relative residue $\frac{||\nabla \chi^2(\vec{\beta})||}{||\nabla \chi^2(\vec{0})||}$')
plt.title('Reconstruction of the spectral indexes using a non-linear PCG')
plt.legend()
plt.show()

In [ ]:
hp.mollview(true_beta, sub=(1,3,1), title='True spectral indexes')
hp.mollview(x_PR_cg, sub=(1,3,2), title='Reconstructed spectral indexes')
hp.mollview(true_beta-x_PR_cg, sub=(1,3,3), title='Difference of the two')

# Fitting the components maps and the spectral indexes all together

In [ ]:
nside = 32
ndetectors = 1
npointings = 8000
nside_beta = 16
frequencies = [140e9, 150e9, 160e9, 200e9, 220e9, 240e9]
npixel = 12*nside**2
nbeta = 12*nside_beta**2

In [ ]:
12*nside**2*1.5/100 * 6 + 12*nside_beta**2*1.5/100

In [ ]:
H_list = []

for freq in frequencies:

    dictname = 'pipeline_demo.dict'
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(dictname)
    d['hwp_stepsize'] = 3
    d['npointings'] = npointings
    d['nside'] = nside
    d['filter_nu'] = freq
    d['synthbeam_kmax'] = 3
    d['synthbeam_fraction'] = 0.95
    d['random_pointing'] = True
    d['repeat_pointing'] = False
    #d['dtheta'] = 180
    q = qubic.QubicInstrument(d)
    s = qubic.QubicScene(d)
    samp = qubic.get_pointing(d)

    acq = qubic.QubicAcquisition(q, samp, s, d)
    H_list.append(acq[:ndetectors].get_operator())

In [ ]:
d['nf_sub']

In [ ]:
skycmb = pysm3.Sky(nside=nside, preset_strings=['c1'], output_unit='uK_CMB')
skydust = pysm3.Sky(nside=nside, preset_strings=['d1'], output_unit='uK_CMB')
skycmb = np.array(skycmb.get_emission(frequencies[-1] * u.Hz))
skydust = np.array(skydust.get_emission(frequencies[-1] * u.Hz))
skydust_beta = pysm3.Sky(nside=nside_beta, preset_strings=['d1'], output_unit='uK_CMB')
true_beta = np.array(skydust_beta.components[0].mbb_index)
true_c = np.concatenate((skycmb[0,:], skycmb[1,:], skycmb[2,:], skydust[0,:], skydust[1,:], skydust[2,:], true_beta))

In [ ]:
coverage = acq[:ndetectors].get_coverage()
seenpix_qubic = coverage/coverage.max() > 0.1
seenpix_qubic_beta = hp.ud_grade(seenpix_qubic, nside_beta)

npixel_patch = np.count_nonzero(seenpix_qubic)
nbeta_patch = np.count_nonzero(seenpix_qubic_beta)

patch_mask = np.concatenate((seenpix_qubic,seenpix_qubic,seenpix_qubic,
                       seenpix_qubic,seenpix_qubic,seenpix_qubic,seenpix_qubic_beta))

def patch_to_sky(c, out):
    sky = true_c.copy()
    sky[patch_mask] = c
    out[...] = sky

Patch_to_Sky = Operator(patch_to_sky, shapein=6*npixel_patch+nbeta_patch, shapeout=6*npixel+nbeta, dtype='float64')

def sky_to_patch(c, out):
    out[...] = c[patch_mask]

Sky_to_Patch = Operator(sky_to_patch, shapein=6*npixel+nbeta, shapeout=6*npixel_patch+nbeta_patch, dtype='float64')

In [ ]:
def modified_black_body(freq, beta):
    nu0 = frequencies[-1]
    return (np.exp(freq * 2.4e-12) - 1) / (np.exp(nu0 * 2.4e-12) - 1) * (freq/nu0)**beta

In [ ]:
def function_A(c, freq, out):
    npixel = 12*nside**2
    power_beta = modified_black_body(freq, c[6*npixel:])
    up_grade_power_beta = hp.ud_grade(power_beta, nside)

    out[:,0] = c[:npixel] + up_grade_power_beta * c[3*npixel:4*npixel]
    out[:,1] = c[npixel:2*npixel] + up_grade_power_beta * c[4*npixel:5*npixel]
    out[:,2] = c[2*npixel:3*npixel] + up_grade_power_beta * c[5*npixel:6*npixel]

A_list = []
for freq in frequencies:
    A_list.append(Operator(lambda c, out, freq=freq : function_A(c, freq, out), 
            shapein=6*12*nside**2 + 12*nside_beta**2, shapeout=(12*nside**2, 3), dtype='float64'))


In [ ]:
# Define the inner operator class
class Transposed_Jacobian(Operator):
    def __init__(self, c, freq, **keywords):
        self.c = c
        self.freq = freq
        self.npixel = 12*nside**2
        self.nbeta = 12*nside_beta**2
        super().__init__(shapein=(self.npixel,3), shapeout=6*self.npixel + self.nbeta, dtype='float64', **keywords)
    
    def direct(self, input_vector, output):
        #print(type(self.c))
        #print(self.c)
        #print(self.c[6*self.npixel:])
        
        power_beta = modified_black_body(self.freq, self.c[6*self.npixel:])
        derive_power_beta = power_beta * np.log(self.freq/frequencies[-1])
        
        output[:self.npixel] = input_vector[:,0]
        output[self.npixel:2*self.npixel] = input_vector[:,1]
        output[2*self.npixel:3*self.npixel] = input_vector[:,2]

        up_grade_power_beta = hp.ud_grade(power_beta, nside)
        output[3*self.npixel:4*self.npixel] = up_grade_power_beta * input_vector[:,0]
        output[4*self.npixel:5*self.npixel] = up_grade_power_beta * input_vector[:,1]
        output[5*self.npixel:6*self.npixel] = up_grade_power_beta * input_vector[:,2]
    
        product = self.c[3*self.npixel:4*self.npixel]*input_vector[:,0] + self.c[4*self.npixel:5*self.npixel]*input_vector[:,1] + self.c[5*self.npixel:6*self.npixel]*input_vector[:,2]
        product = hp.ud_grade(product, nside_beta) * (self.npixel // self.nbeta)
        
        output[6*self.npixel:] = derive_power_beta * product

# Define the outer operator class
class Generate_Transposed_Jacobian(Operator):
    def direct(self, c, freq, output):
        # Create the generated operator
        transposed_jacobian = Transposed_Jacobian(c, freq)
        # Store the generated operator in the output
        output[...] = transposed_jacobian

# Initialize the outer operator
generate_transposed_jacobian = Generate_Transposed_Jacobian()


In [ ]:
d = H_list[0](A_list[0](true_c))
for i in range(1, len(frequencies)):
    d = d + (H_list[i](A_list[i](true_c)))

In [ ]:
invN = acq[:ndetectors].get_invntt_operator()

In [ ]:
def grad_operator(c, out):
    W = H_list[0](A_list[0](Patch_to_Sky(c)))
    for i in range(1, len(frequencies)):
        W = W + (H_list[i](A_list[i](Patch_to_Sky(c))))
    W = invN(W - d)
    #hp.mollview(HT_list[0](W)[:,0], max=1e-36, min=-1e-34)

    output_operator = np.empty((), dtype=object)
    generate_transposed_jacobian.direct(Patch_to_Sky(c), frequencies[0], output_operator)
    transposed_jacobian = output_operator.item()
    X = transposed_jacobian(H_list[0].T(W))
    for i in range(1, len(frequencies)):
        generate_transposed_jacobian.direct(Patch_to_Sky(c), frequencies[i], output_operator)
        transposed_jacobian = output_operator.item()
        X = X + transposed_jacobian(H_list[i].T(W))
    
    out[...] = Sky_to_Patch(X)

grad_chi_squared = Operator(grad_operator, shapein=6*npixel_patch+nbeta_patch, 
                            shapeout=6*npixel_patch+nbeta_patch, dtype='float64')

In [ ]:
(grad_chi_squared(Sky_to_Patch(true_c))==0).all()

In [ ]:
Cov = np.empty((len(frequencies), 3*npixel_patch))
mixed_map_mask = np.concatenate([seenpix_qubic,seenpix_qubic,seenpix_qubic])

for i in range(3*npixel_patch):
    patch_vector = np.zeros((npixel_patch,3))
    patch_vector[i%npixel_patch, i//npixel_patch] = 1
    basis_vector = np.zeros((npixel,3))
    basis_vector[seenpix_qubic, :] = patch_vector
    for freq_index in range(len(frequencies)):
        vector_i = H_list[freq_index](basis_vector)
        vector_i = vector_i.ravel()
        Cov[freq_index, i] = np.dot(vector_i, vector_i)
    patch_vector[i%npixel_patch, i//npixel_patch] = 0

In [ ]:
def hessian_inverse_diagonal(c, out):
    sky_c = Patch_to_Sky(c)
    dust_spectrum_squared = np.zeros((len(frequencies),npixel_patch))
    derive_dust_spectrum_squared = np.zeros((len(frequencies),npixel_patch))
    for index, freq in enumerate(frequencies):
        dust_spectrum = hp.ud_grade(modified_black_body(freq, sky_c[6*npixel:]), nside)[seenpix_qubic]
        dust_spectrum_squared[index,:] = dust_spectrum**2
        derive_dust_spectrum_squared[index,:] = (dust_spectrum * np.log(freq/frequencies[-1]))**2
    
    out[:3*npixel_patch] = 1/np.sum(Cov, axis=0)
    out[3*npixel_patch:6*npixel_patch] = 1/np.sum(np.concatenate((dust_spectrum_squared,
                                                                dust_spectrum_squared,dust_spectrum_squared),1)*Cov, axis=0)

    factor = np.sum(np.concatenate((derive_dust_spectrum_squared,
                                   derive_dust_spectrum_squared,derive_dust_spectrum_squared),1)*Cov, axis=0)
    diagonal_beta = factor[:npixel_patch]*c[3*npixel_patch:4*npixel_patch]**2
    diagonal_beta += factor[npixel_patch:2*npixel_patch]*c[4*npixel_patch:5*npixel_patch]**2
    diagonal_beta += factor[2*npixel_patch:3*npixel_patch]*c[5*npixel_patch:6*npixel_patch]**2
    downgrader = np.zeros(npixel)
    downgrader[seenpix_qubic] = diagonal_beta
    downgrader = hp.ud_grade(downgrader, nside_beta)*(npixel/nbeta)
    out[6*npixel_patch:] = 1/downgrader[seenpix_qubic_beta]

    out /= np.mean(out)

HessianInverseDiagonal = Operator(hessian_inverse_diagonal, shapein=6*npixel_patch+nbeta_patch, shapeout=6*npixel_patch+nbeta_patch, dtype='float64')

M = DiagonalOperator(HessianInverseDiagonal(Sky_to_Patch(true_c)))



In [ ]:
max_iteration = 1000
x0 = np.zeros(6*npixel_patch+nbeta_patch)
#x0[3*npixel:6*npixel] = skydust.ravel()

x0[:npixel_patch] = Sky_to_Patch(true_c)[:npixel_patch].copy()
x0[npixel_patch:3*npixel_patch] = np.zeros(2*npixel_patch)
x0[3*npixel_patch:4*npixel_patch] = Sky_to_Patch(true_c)[3*npixel_patch:4*npixel_patch].copy()
x0[4*npixel_patch:6*npixel_patch] = 0*Sky_to_Patch(true_c)[4*npixel_patch:6*npixel_patch]*(1+np.random.normal(scale=1.0, size=2*npixel_patch))
x0[6*npixel_patch:] = np.ones(nbeta_patch)*1.54


#x0[6*npixel_patch:] = Sky_to_Patch(true_c)[6*npixel_patch:]
#x0 = true_c.copy()
#x0[:npixel] = np.zeros(npixel)

sigma=1e-3

residues_PR_cg = []

start = time()
pcg = non_linear_pcg(grad_chi_squared, conjugate_method='polak-ribiere', x0=x0, M=M, tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=max_iteration, residues=residues_PR_cg)
x_PR_cg = pcg['x']
residues_PR_cg /= np.linalg.norm(grad_chi_squared(x0))
print(f'time for PR CG: {time()-start}')


In [ ]:
residues_hybrid_cg = []

start = time()
pcg = non_linear_pcg(grad_chi_squared, conjugate_method='hybrid', x0=x0, M=M, tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=max_iteration, residues=residues_hybrid_cg)
x_hybrid_cg = pcg['x']
residues_hybrid_cg /= np.linalg.norm(grad_chi_squared(x0))
print(f'time for hybrid CG: {time()-start}')


In [ ]:
plt.plot(residues_PR_cg, label='Polak-Ribière CG')
plt.plot(residues_hybrid_cg, label='Hybrid CG')
plt.yscale('log')
plt.grid(axis='y', linestyle='dotted')
plt.xlabel('Number of iterations')
plt.ylabel(r'Relative residue $\frac{||\nabla \chi^2(\vec{\beta})||}{||\nabla \chi^2(\vec{0})||}$')
plt.title('Simultaneous reconstruction of components maps and spectral indices using a non-linear PCG')
plt.legend()
plt.show()

In [ ]:
x = Patch_to_Sky(x_PR_cg)
x0sky = Patch_to_Sky(x0)
name_list = ['CMB I','CMB Q','CMB U','dust I','dust Q','dust U',r'$\beta$']

plt.figure(figsize=(12, 25))
for i in range(6):
    hp.gnomview(true_c[i*npixel:(i+1)*npixel], sub=(7,4,4*i+1), title='True '+name_list[i], rot=qubic.equ2gal(0, -57),reso=18, cmap='jet')
    hp.gnomview(x0sky[i*npixel:(i+1)*npixel], sub=(7,4,4*i+2), title='Initial '+name_list[i], rot=qubic.equ2gal(0, -57),reso=18, cmap='jet')
    hp.gnomview(x[i*npixel:(i+1)*npixel], sub=(7,4,4*i+3), title='Reconstructed '+name_list[i], rot=qubic.equ2gal(0, -57),reso=18, cmap='jet')
    r = true_c[i*npixel:(i+1)*npixel] - x[i*npixel:(i+1)*npixel]
    sig = np.std(r[seenpix_qubic])
    hp.gnomview(r, sub=(7,4,4*i+4), title='Difference '+name_list[i], rot=qubic.equ2gal(0, -57),reso=18, min=-2*sig, max=2*sig, cmap='jet')
hp.gnomview(true_c[6*npixel:], sub=(7,4,4*6+1), title='True '+name_list[6], rot=qubic.equ2gal(0, -57),reso=18, cmap='jet')
hp.gnomview(x0sky[6*npixel:], sub=(7,4,4*6+2), title='Initial '+name_list[6], rot=qubic.equ2gal(0, -57),reso=18, cmap='jet')
hp.gnomview(x[6*npixel:], sub=(7,4,4*6+3), title='Reconstructed '+name_list[6], rot=qubic.equ2gal(0, -57),reso=18, cmap='jet')
r = true_c[6*npixel:] - x[6*npixel:]
sig = np.std(r[seenpix_qubic_beta])
hp.gnomview(r, sub=(7,4,4*6+4), title='Difference '+name_list[6], rot=qubic.equ2gal(0, -57),reso=18, min=-2*sig, max=2*sig, cmap='jet')
plt.show()


In [ ]:
plt.plot(1/HessianInverseDiagonal(Sky_to_Patch(true_c)))
plt.yscale('log')
plt.show()

In [ ]:
epsilon = 1e-5
hessian = np.empty((6*npixel_patch+nbeta_patch, 6*npixel_patch+nbeta_patch))
ei = np.zeros(6*npixel_patch+nbeta_patch)
for i in range(6*npixel_patch+nbeta_patch):
    ei[i-1]=0
    ei[i]=epsilon
    hessian[i,:] = grad_chi_squared(Sky_to_Patch(true_c)+ei)/epsilon
    

In [ ]:
plt.matshow(hessian, norm='log')#[:50,:50], norm='log', vmin=1e-2)#[:50,:50])
plt.colorbar()

In [ ]:
factor = np.zeros((len(frequencies),npixel_patch))
for index, freq in enumerate(frequencies):
    factor[index,:] = hp.ud_grade(modified_black_body(freq, true_c[6*npixel:])**2, nside)[seenpix_qubic]

In [ ]:
Cov = np.empty((len(frequencies), 3*npixel_patch))
mixed_map_mask = np.concatenate([seenpix_qubic,seenpix_qubic,seenpix_qubic])

for i in range(3*npixel_patch):
    patch_vector = np.zeros((npixel_patch,3))
    patch_vector[i%npixel_patch, i//npixel_patch] = 1
    basis_vector = np.zeros((npixel,3))
    basis_vector[seenpix_qubic, :] = patch_vector
    for freq_index in range(len(frequencies)):
        vector_i = H_list[freq_index](basis_vector)
        vector_i = vector_i.ravel()
        Cov[freq_index, i] = np.dot(vector_i, vector_i)
    patch_vector[i%npixel_patch, i//npixel_patch] = 0

In [ ]:
def hessian_inverse_diagonal(c, out):
    sky_c = Patch_to_Sky(c)
    dust_spectrum_squared = np.zeros((len(frequencies),npixel_patch))
    derive_dust_spectrum_squared = np.zeros((len(frequencies),npixel_patch))
    for index, freq in enumerate(frequencies):
        dust_spectrum = hp.ud_grade(modified_black_body(freq, sky_c[6*npixel:]), nside)[seenpix_qubic]
        dust_spectrum_squared[index,:] = dust_spectrum**2
        derive_dust_spectrum_squared[index,:] = (dust_spectrum * np.log(freq/frequencies[-1]))**2
    
    out[:3*npixel_patch] = 1/np.sum(Cov, axis=0)
    out[3*npixel_patch:6*npixel_patch] = 1/np.sum(np.concatenate((dust_spectrum_squared,
                                                                dust_spectrum_squared,dust_spectrum_squared),1)*Cov, axis=0)

    factor = np.sum(np.concatenate((derive_dust_spectrum_squared,
                                   derive_dust_spectrum_squared,derive_dust_spectrum_squared),1)*Cov, axis=0)
    diagonal_beta = factor[:npixel_patch]*c[3*npixel_patch:4*npixel_patch]**2
    diagonal_beta += factor[npixel_patch:2*npixel_patch]*c[4*npixel_patch:5*npixel_patch]**2
    diagonal_beta += factor[2*npixel_patch:3*npixel_patch]*c[5*npixel_patch:6*npixel_patch]**2
    downgrader = np.zeros(npixel)
    downgrader[seenpix_qubic] = diagonal_beta
    downgrader = hp.ud_grade(downgrader, nside_beta)*(npixel/nbeta)
    out[6*npixel_patch:] = 1/downgrader[seenpix_qubic_beta]

    out /= np.mean(out)

HessianInverseDiagonal = Operator(hessian_inverse_diagonal, shapein=6*npixel_patch+nbeta_patch, shapeout=6*npixel_patch+nbeta_patch, dtype='float64')

M = DiagonalOperator(HessianInverseDiagonal(Sky_to_Patch(true_c)))



In [ ]:
M(np.ones(6*npixel_patch+nbeta_patch))

In [ ]:
HessianInverseDiagonal(Sky_to_Patch(true_c))

In [ ]:
plt.plot(np.diag(hessian)*HessianInverseDiagonal(Sky_to_Patch(true_c)))
#plt.yscale('log')


In [ ]:
plt.plot(np.diag(hessian)[:3*npixel_patch]/np.concatenate((Cov, np.concatenate((factor,factor,factor))*Cov))[:3*npixel_patch])
#plt.yscale('log')

In [ ]:
plt.plot(np.diag(hessian))
plt.yscale('log')

In [ ]:
plt.plot(coverage[seenpix_qubic])
plt.yscale('log')

In [ ]:
plt.matshow(hessian[:npixel_patch,:npixel_patch], norm='log')
plt.colorbar()
plt.matshow(hessian[npixel_patch:2*npixel_patch,npixel_patch:2*npixel_patch], norm='log')
plt.colorbar()
plt.matshow(hessian[6*npixel_patch:,6*npixel_patch:], norm='log')
plt.colorbar()

In [ ]:
epsilon = 1e-10
diag = grad_chi_squared(Sky_to_Patch(true_c)+epsilon)/epsilon
plt.plot(diag[:6*npixel_patch])
#plt.yscale('log')

# Fitting the spectral indices

In [ ]:
nside = 128
ndetectors = 1
npointings = 8000
nside_beta = 32
frequencies = [140e9, 150e9, 160e9, 200e9, 220e9, 240e9]
npixel = 12*nside**2
nbeta = 12*nside_beta**2

In [ ]:
skycmb = pysm3.Sky(nside=nside, preset_strings=['c1'], output_unit='uK_CMB')
skydust = pysm3.Sky(nside=nside, preset_strings=['d1'], output_unit='uK_CMB')
skycmb = np.array(skycmb.get_emission(frequencies[-1] * u.Hz))
skydust = np.array(skydust.get_emission(frequencies[-1] * u.Hz))
skydust_beta = pysm3.Sky(nside=nside_beta, preset_strings=['d1'], output_unit='uK_CMB')
true_beta = np.array(skydust_beta.components[0].mbb_index)
true_c = np.concatenate((skycmb[0,:], skycmb[1,:], skycmb[2,:], skydust[0,:], skydust[1,:], skydust[2,:], true_beta))
#true_c[3*npixel:6*npixel] = np.ones(3*npixel)

In [ ]:
H_list = []

for freq in frequencies:

    dictname = 'pipeline_demo.dict'
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(dictname)
    d['hwp_stepsize'] = 3
    d['npointings'] = npointings
    d['nside'] = nside
    d['filter_nu'] = freq
    d['synthbeam_kmax'] = 3
    d['synthbeam_fraction'] = 0.95
    d['random_pointing'] = True
    d['repeat_pointing'] = False
    #d['dtheta'] = 180
    q = qubic.QubicInstrument(d)
    s = qubic.QubicScene(d)
    samp = qubic.get_pointing(d)

    acq = qubic.QubicAcquisition(q, samp, s, d)
    H_list.append(acq[:ndetectors].get_operator())

In [ ]:
coverage = acq[:ndetectors].get_coverage()
seenpix_qubic = coverage/coverage.max() > 0.1
seenpix_qubic_beta = hp.ud_grade(seenpix_qubic, nside_beta)

npixel_patch = np.count_nonzero(seenpix_qubic)
nbeta_patch = np.count_nonzero(seenpix_qubic_beta)

patch_mask = np.concatenate((seenpix_qubic,seenpix_qubic,seenpix_qubic,
                       seenpix_qubic,seenpix_qubic,seenpix_qubic,seenpix_qubic_beta))

def patch_to_sky(c, out):
    sky = true_c[6*npixel:].copy()
    sky[seenpix_qubic_beta] = c
    out[...] = sky

Patch_to_Sky = Operator(patch_to_sky, shapein=nbeta_patch, shapeout=nbeta, dtype='float64')

def sky_to_patch(c, out):
    out[...] = c[seenpix_qubic_beta]

Sky_to_Patch = Operator(sky_to_patch, shapein=nbeta, shapeout=nbeta_patch, dtype='float64')

In [ ]:
def modified_black_body(freq, beta):
    nu0 = frequencies[-1]
    return (np.exp(freq * 2.4e-18) - 1) / (np.exp(nu0 * 2.4e-18) - 1) * (freq/nu0)**beta

In [ ]:
def function_A(c, freq, out):
    power_beta = modified_black_body(freq, c)
    up_grade_power_beta = hp.ud_grade(power_beta, nside)

    out[:,0] = true_c[:npixel] + up_grade_power_beta * true_c[3*npixel:4*npixel]
    out[:,1] = true_c[npixel:2*npixel] + up_grade_power_beta * true_c[4*npixel:5*npixel]
    out[:,2] = true_c[2*npixel:3*npixel] + up_grade_power_beta * true_c[5*npixel:6*npixel]

A_list = []
for freq in frequencies:
    A_list.append(Operator(lambda c, out, freq=freq : function_A(c, freq, out), 
            shapein=12*nside_beta**2, shapeout=(12*nside**2, 3), dtype='float64'))


In [ ]:
# Define the inner operator class
class Transposed_Jacobian(Operator):
    def __init__(self, c, freq, **keywords):
        self.c = c
        self.freq = freq
        self.npixel = 12*nside**2
        self.nbeta = 12*nside_beta**2
        super().__init__(shapein=(self.npixel,3), shapeout=self.nbeta, dtype='float64', **keywords)
    
    def direct(self, input_vector, output):
        power_beta = modified_black_body(self.freq, self.c)
        derive_power_beta = power_beta * np.log(self.freq/frequencies[-1])
    
        product = true_c[3*self.npixel:4*self.npixel]*input_vector[:,0] + true_c[4*self.npixel:5*self.npixel]*input_vector[:,1] + true_c[5*self.npixel:6*self.npixel]*input_vector[:,2]
        product = hp.ud_grade(product, nside_beta) * (self.npixel // self.nbeta)
        
        output[...] = derive_power_beta * product

# Define the outer operator class
class Generate_Transposed_Jacobian(Operator):
    def direct(self, c, freq, output):
        # Create the generated operator
        transposed_jacobian = Transposed_Jacobian(c, freq)
        # Store the generated operator in the output
        output[...] = transposed_jacobian

# Initialize the outer operator
generate_transposed_jacobian = Generate_Transposed_Jacobian()


In [ ]:
d = H_list[0] * A_list[0](true_c[6*npixel:])
for i in range(1, len(frequencies)):
    d = d + (H_list[i] * A_list[i](true_c[6*npixel:]))

In [ ]:
invN = acq[:ndetectors].get_invntt_operator()

In [ ]:
def grad_operator(c, out):
    W = H_list[0] * A_list[0](Patch_to_Sky(c))
    for i in range(1, len(frequencies)):
        W += (H_list[i] * A_list[i](Patch_to_Sky(c)))
    W = invN * W - (invN(d))

    output_operator = np.empty((), dtype=object)
    generate_transposed_jacobian.direct(Patch_to_Sky(c), frequencies[0], output_operator)
    transposed_jacobian = output_operator.item()
    X = transposed_jacobian(H_list[0].T * W)
    for i in range(1, len(frequencies)):
        generate_transposed_jacobian.direct(Patch_to_Sky(c), frequencies[i], output_operator)
        transposed_jacobian = output_operator.item()
        X = X + transposed_jacobian(H_list[i].T * W)
    
    out[...] = Sky_to_Patch(X)

grad_chi_squared = Operator(grad_operator, shapein=nbeta_patch, 
                            shapeout=nbeta_patch, dtype='float64')

In [ ]:
max_iteration = 100
#x0 = np.zeros(true_c.shape)
#x0[3*npixel:6*npixel] = skydust.ravel()
#x0[6*npixel:] = np.ones(nbeta)*1.54
x0 = np.ones(nbeta_patch) *1.5

sigma=1e-2

residues_PR_cg = []

start = time()
pcg = non_linear_pcg(grad_chi_squared, conjugate_method='polak-ribiere', x0=x0, tol=1e-16, sigma_0=sigma, tol_linesearch=1e-3, maxiter=max_iteration, residues=residues_PR_cg)
x_PR_cg = pcg['x']
residues_PR_cg /= np.linalg.norm(grad_chi_squared(x0))
print(f'time for PR CG: {time()-start}')


In [ ]:
plt.plot(residues_PR_cg, label='Polak-Ribière CG')
plt.yscale('log')
plt.grid(axis='y', linestyle='dotted')
plt.xlabel('Number of iterations')
plt.ylabel(r'Relative residue $\frac{||\nabla \chi^2(\vec{\beta})||}{||\nabla \chi^2(\vec{0})||}$')
plt.title('Reconstruction of the spectral indices using a non-linear PCG')
plt.legend()
plt.show()

In [ ]:
hp.mollview(true_c[6*npixel:], sub=(1,3,1))
hp.mollview(Patch_to_Sky(x_PR_cg), sub=(1,3,2))
hp.mollview(true_c[6*npixel:] - Patch_to_Sky(x_PR_cg), sub=(1,3,3))
plt.show()

In [ ]:
nbeta_patch

In [ ]:
c = np.zeros(nbeta_patch)
c[40]=1
hp.gnomview(true_c[3*npixel:4*npixel], rot=qubic.equ2gal(0, -57),reso=15)
hp.gnomview(true_c[6*npixel:], rot=qubic.equ2gal(0, -57),reso=15)
hp.gnomview(Patch_to_Sky(c), rot=qubic.equ2gal(0, -57),reso=15)

In [ ]:
epsilon = 1e-3
hessian = np.empty((nbeta_patch, nbeta_patch))
for i in range(nbeta_patch):
    ei = np.zeros(nbeta_patch)
    ei[i]=1
    hessian[i,:] = grad_chi_squared(Sky_to_Patch(true_c[6*npixel:])+epsilon*ei)/epsilon
    

In [ ]:
plt.matshow(hessian[:50,:50], norm='log', vmin=1e-2)#[:50,:50])
plt.colorbar()

In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh(hessian)

In [ ]:
plt.plot(eigenvalues[::-1])
plt.yscale('log')

In [ ]:
M=np.linalg.inv(hessian)

In [ ]:
(grad_chi_squared(true_c[6*npixel:])==0)#.all()

In [ ]:
grad_chi_squared(true_c[6*npixel:])

In [ ]:
d

In [ ]:
d